# Вебинар 4. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

## Задание 1. Напишите свою функцию prefilter_items

In [29]:
from config import pseudo_item_id

In [31]:
data_train.shape

(2278490, 12)

In [70]:
def prefilter_items(data, take_n_popular=5000, popular_col='quantity', min_value=1, max_value=30, week_left=52):
    """Предфильтрация товаров"""
    data2 = data.copy()
    
    # 1. Удаление товаров, со средней ценой < 1$
    data2.loc[data2['sales_value']/data2['quantity'] < min_value, 'item_id'] = pseudo_item_id
#     data2 = data2.loc[data2['sales_value']/data2['quantity'] > min_value]
    
    # 2. Удаление товаров со соедней ценой > 30$
    data2.loc[data2['sales_value']/data2['quantity'] > max_value, 'item_id'] = pseudo_item_id
#     data2 = data2.loc[data2['sales_value']/data2['quantity'] < max_value]
    
    # 3. Придумайте свой фильтр
    # 3.1 Убираем непонятную положительную скидку
    data2.loc[data2['retail_disc'] > 0, 'item_id'] = pseudo_item_id
#     data2 = data2.loc[data2['retail_disc'] <= 0]
    
    # 3.2 Оставляем только товары, купленные в последние N недель:
    from_week = data2['week_no'].max() - week_left
    data2 = data2.loc[data2['week_no'] > from_week]
    
    # 4. Выбор топ-N самых популярных товаров (N = take_n_popular)    
    popularity = data2.groupby('item_id')[popular_col].sum().reset_index()
    top_items = popularity.sort_values(popular_col, ascending=False).head(take_n_popular).item_id.tolist()
    data2.loc[~data2['item_id'].isin(top_items), 'item_id'] = pseudo_item_id
    
    return data2

В корне проекта создайте модуль *src*. Положите функцию *prefilter_items* в файл *src/utils.py*  

Далее запустите код ниже и перейдите к заданию 2 (в конце ноутбука)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items, get_ctm

In [3]:
data = pd.read_csv('./raw_data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Warm start
train_items = data_train['item_id'].unique()
train_users = data_train['user_id'].unique()

data_test = data_test[data_test['item_id'].isin(train_items)]
data_test = data_test[data_test['user_id'].isin(train_users)]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [10]:
data_train.describe()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
count,2.278490e+06,2.278490e+06,2.278490e+06,2.278490e+06,2.278490e+06,2.278490e+06,2.278490e+06,2.278490e+06,2.278490e+06,2.278490e+06,2.278490e+06,2.278490e+06
mean,1.271764e+03,3.294526e+10,3.491402e+02,2.791955e+06,1.006171e+02,3.095110e+00,2.992061e+03,-5.393603e-01,1.562467e+03,5.056328e+01,-1.646478e-02,-2.915685e-03
std,7.269816e+02,3.964679e+09,1.676271e+02,3.673791e+06,1.153002e+03,4.196106e+00,8.693638e+03,1.236080e+00,4.025741e+02,2.394798e+01,2.179563e-01,3.995998e-02
min,1.000000e+00,2.698485e+10,1.000000e+00,2.567100e+04,0.000000e+00,0.000000e+00,1.000000e+00,-1.300200e+02,0.000000e+00,1.000000e+00,-5.593000e+01,-7.700000e+00
25%,6.540000e+02,3.003546e+10,2.080000e+02,9.167670e+05,1.000000e+00,1.270000e+00,3.300000e+02,-6.900000e-01,1.306000e+03,3.000000e+01,0.000000e+00,0.000000e+00
50%,1.271000e+03,3.214976e+10,3.510000e+02,1.027068e+06,1.000000e+00,2.000000e+00,3.700000e+02,-2.000000e-02,1.615000e+03,5.100000e+01,0.000000e+00,0.000000e+00
75%,1.914000e+03,3.433825e+10,4.940000e+02,1.131351e+06,1.000000e+00,3.490000e+00,4.220000e+02,0.000000e+00,1.846000e+03,7.100000e+01,0.000000e+00,0.000000e+00
max,2.500000e+03,4.129777e+10,6.350000e+02,1.782923e+07,8.963800e+04,8.400000e+02,3.428000e+04,3.990000e+00,2.359000e+03,9.100000e+01,0.000000e+00,0.000000e+00


In [11]:
data_train.groupby('item_id')[['quantity', 'sales_value']].sum().reset_index()

,item_id,quantity,sales_value
0,25671,6,20.94
1,26081,1,0.99
2,26093,1,1.59
3,26190,1,1.54
4,26355,2,1.98
...,...,...,...
86860,17381856,0,0.00
86861,17382205,1,7.99
86862,17383227,1,4.49
86863,17827644,1,2.50


In [4]:
top_col = 'quantity'
top_n = 5000
data_train = prefilter_items(data_train,
                                  take_n_popular=top_n, popular_col=top_col,
                                  min_value=1, max_value=30, week_left=52)
data_train['item_id'].nunique()

5001

In [5]:
data_train.shape

(2278490, 12)

In [6]:
item_features = pd.read_csv('./raw_data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [10]:
len(item_features.loc[item_features['brand']=='Private', 'item_id'].to_list())

13816

In [9]:
ctm_df = get_ctm(item_features)
len(ctm_df)

13816

In [11]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [32]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471,999999999
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [33]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [34]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [35]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=False)


Wall time: 760 ms


In [36]:
[id_to_itemid[rec[0]] for rec in recs]

[866227, 995785, 1029743, 1110764, 999999999]

In [19]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [138]:
%%time

model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


CPU times: user 8.35 s, sys: 1.08 s, total: 9.42 s
Wall time: 2.44 s


-----

# Домашнее задание

1. Перенесите метрики из ДЗ 1 src/metrics.py

Готово

2. Создайте модуль src/recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src/recommenders.py

Готово

3. Проверьте, что все модули корректно импортируются

In [37]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """
    
    def __init__(self, data, ctm, top_col='quantity', weighting=True):
        """
        data - датафрейм с покупками юзер - товар
        ctm - словарь принадлежности товара к СТМ {item_id: 0/1}
        top_col - фича, по которой будем определять популярность товаров
        """
                
        # Топ покупок каждого юзера
        self.top_purchases = data.groupby(['user_id', 'item_id'])[top_col].count().reset_index()
        self.top_purchases.sort_values(top_col, ascending=False, inplace=True)
        self.top_purchases = self.top_purchases[self.top_purchases['item_id'] != pseudo_item_id]

        # Топ покупок по всему датасету
        self.overall_top_purchases = data.groupby('item_id')[top_col].count().reset_index()
        self.overall_top_purchases.sort_values(top_col, ascending=False, inplace=True)
        self.overall_top_purchases = self.overall_top_purchases[self.overall_top_purchases['item_id'] != pseudo_item_id]
        self.overall_top_purchases = self.overall_top_purchases.item_id.tolist()
        
        self.user_item_matrix = self.prepare_matrix(data, top_col)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = self.prepare_dicts(self.user_item_matrix)
        
        # Словарь {item_id: 0/1}. 0/1 - факт принадлежности товара к СТМ
        self.item_id_to_ctm = ctm
        
        # Own recommender обучается до взвешивания матрицы
        self.own_recommender = self.fit_own_recommender()
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit()

    @staticmethod
    def prepare_matrix(data, top_col):
        if top_col == 'sales_value':
            aggfunc = 'sum'
        else:
            aggfunc = 'count'
            
            
        user_item_matrix = pd.pivot_table(data, 
                                  index='user_id', columns='item_id', 
                                  values=top_col, # Можно пробовать другие варианты
                                  aggfunc=aggfunc, 
                                  fill_value=0
                                 )
        
        user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
        user_item_matrix = user_item_matrix.astype(float)
        
        return user_item_matrix

    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    def fit_own_recommender(self):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(self.user_item_matrix).T.tocsr(), show_progress=False)
        
        return own_recommender

    def fit(self, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=n_factors,
                                         regularization=regularization,
                                         iterations=iterations,
                                         num_threads=num_threads)
        model.fit(csr_matrix(self.user_item_matrix).T.tocsr(), show_progress=False)
        
        return model

    def get_similar_items_recommendation(self, user, filter_ctm=True, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        # your_code
        # Практически полностью реализовали на прошлом вебинаре
        # Не забывайте, что нужно учесть параметр filter_ctm
        sparse_user_item = csr_matrix(self.user_item_matrix).tocsr()
        filter_items = [self.itemid_to_id[pseudo_item_id]]
        # filter_items = None
        
        if filter_ctm:
            filter_items.expand(self.item_id_to_ctm)
        
        res = [self.id_to_itemid[rec[0]] for rec in 
                    self.own_recommender.recommend(userid=self.userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=True,
                                    filter_items=filter_items,
                                    recalculate_user=True)]

        return res

    def get_similar_users_recommendation(self, user, filter_ctm=True, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
        sparse_user_item = csr_matrix(self.user_item_matrix).tocsr()
        filter_items = [self.itemid_to_id[pseudo_item_id]]
        
        res = [self.id_to_itemid[rec[0]] for rec in 
                    self.model.recommend(userid=self.userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=filter_items, 
                                    recalculate_user=True)]

        return res

----

Проверка, что все работает

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.utils import prefilter_items, get_ctm, Result
from src.train_test_split import get_train_test
from src.recommenders import MainRecommender

In [3]:
def load_data_user_item(path_to_data):
    data = pd.read_csv(path_to_data)

    data.columns = [col.lower() for col in data.columns]
    data.rename(columns={'household_key': 'user_id',
                        'product_id': 'item_id'},
               inplace=True)

    return data


def load_item_features(path_to_data):
    data = pd.read_csv('./raw_data/product.csv')
    data.columns = [col.lower() for col in data.columns]
    data.rename(columns={'product_id': 'item_id'}, inplace=True)

    return data

In [4]:
data_user_item_path = './raw_data/retail_train.csv'
data_item_features = './raw_data/product.csv'

df = load_data_user_item(data_user_item_path)
data_train, data_test = get_train_test(df)

In [54]:
data_train.shape, data_test.shape

((2278490, 12), (115644, 12))

In [6]:
item_features = load_item_features(data_item_features)

In [7]:
ctm_res = get_ctm(item_features)
len(ctm_res)

13816

In [8]:
top_col = 'quantity'
top_n = 5000
data_train2 = prefilter_items(data_train,
                              take_n_popular=top_n, popular_col=top_col,
                              min_value=1, max_value=30, week_left=52)
data_train['item_id'].nunique(), data_train2['item_id'].nunique()

(86865, 5001)

In [9]:
data_train2.shape

(2278490, 12)

In [10]:
%%time
recommender = MainRecommender(data_train2, ctm_res, top_col=top_col, weighting=False)

Wall time: 10.1 s


In [11]:
result = Result(data_test)
result.get_result().head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [13]:
%%time
result.add_result('own_purchases_rec', recommender.get_similar_items_recommendation)

Wall time: 6min 48s


True

In [14]:
%%time
result.add_result('similar_items_rec', recommender.get_similar_users_recommendation)

Wall time: 8min 24s


True

In [15]:
%%time
result.count_presision('own_purchases_rec')

Wall time: 200 ms


True

In [16]:
%%time
result.count_presision('similar_items_rec')

Wall time: 145 ms


True

In [17]:
print('own_purchases_rec precision_at_k', result.get_precision_at_k('own_purchases_rec'))
print('similar_items_rec precision_at_k', result.get_precision_at_k('similar_items_rec'))

own_purchases_rec precision_at_k 0.146411092985318
similar_items_rec precision_at_k 0.20578147966682717
Wall time: 997 µs


In [23]:
%%time
recommender = MainRecommender(data_train2, ctm_res, top_col=top_col, weighting=True)

weighting done
Wall time: 10.4 s


In [24]:
%%time
result.add_result('own_purchases_rec_weight', recommender.get_similar_items_recommendation)

Wall time: 15.6 s


True

In [25]:
%%time
result.add_result('similar_items_rec_weight', recommender.get_similar_users_recommendation)

Wall time: 29.5 s


True

In [20]:
%%time
result.count_presision('own_purchases_rec_weight')
result.count_presision('similar_items_rec_weight')

Wall time: 313 ms


True

In [28]:
print('own_purchases_rec_weight precision_at_k', result.get_precision_at_k('own_purchases_rec_weight'))
print('similar_items_rec_weight precision_at_k', result.get_precision_at_k('similar_items_rec_weight'))

own_purchases_rec_weight precision_at_k 0.146411092985318
similar_items_rec_weight precision_at_k 0.17197452229299182
Wall time: 998 µs


#### С предфильтрацией 1$ < q/s_v < 30$

In [53]:
data_train3 = prefilter_items(data_train)
data_train3['item_id'].nunique()

5000

In [64]:
%%time
recommender = MainRecommender(data_train3, ctm_res, top_col=top_col, weighting=True)

weighting done
Wall time: 10.7 s


In [65]:
%%time
result.add_result('own_purchases_rec_weight_pre1_30', recommender.get_similar_items_recommendation)

Wall time: 13.4 s


True

In [66]:
%%time
result.add_result('similar_items_rec_weight_pre1_30', recommender.get_similar_users_recommendation)

Wall time: 28.8 s


True

In [67]:
%%time
result.count_presision('own_purchases_rec_weight_pre1_30')
result.count_presision('similar_items_rec_weight_pre1_30')

Wall time: 319 ms


True

In [68]:
print('own_purchases_rec_weight_pre1_30 precision_at_k', result.get_precision_at_k('own_purchases_rec_weight_pre1_30'))
print('similar_items_rec_weight_pre1_30 precision_at_k', result.get_precision_at_k('similar_items_rec_weight_pre1_30'))

own_purchases_rec_weight_pre1_30 precision_at_k nan
similar_items_rec_weight_pre1_30 precision_at_k 0.14551690347868565
Wall time: 999 µs


In [74]:
result.get_result().head(2)

,user_id,actual,own_purchases_rec,similar_items_rec,own_purchases_rec_weight,similar_items_rec_weight,own_purchases_rec_pre1_30,similar_items_rec_pre1_30,own_purchases_rec_weight_pre1_30,similar_items_rec_weight_pre1_30
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[866227, 995785, 1029743, 1110764]","[1082185, 995242, 1127831, 981760, 1033142]","[866227, 995785, 1029743, 1110764]","[1082185, 1075074, 932949, 1019643, 960732]",[],"[965766, 995242, 981760, 1127831, 1082185]",[],"[960732, 1075074, 8090541, 904777, 1102067]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1023958, 8205731]","[951590, 5569230, 1106523, 826249, 910032]","[1023958, 8205731]","[826249, 1044078, 883404, 1098066, 951590]",[],"[951590, 908531, 1106523, 1044078, 5569230]",[],"[1044078, 844179, 859075, 5568378, 892008]"


У меня не выдаются рекомендации собственных покупок, если я включаю предфильтрацию по стоимости товара. Не понимаю, в чем причина? Буду признателен, если подскажите, где проблема?

#### С полной предфильтрацией 

In [71]:
data_train4 = prefilter_items(data_train)
data_train4['item_id'].nunique()

5000

In [72]:
%%time
recommender = MainRecommender(data_train4, ctm_res, top_col=top_col, weighting=True)

weighting done
Wall time: 9.6 s


In [73]:
%%time
result.add_result('own_purchases_rec_weight_pre_full', recommender.get_similar_items_recommendation)

KeyError: 60

In [75]:
%%time
result.add_result('similar_items_rec_weight_pre_full', recommender.get_similar_users_recommendation)

KeyError: 60

Если делаю полную предфильтрацию, т.е. еще и обрезаю данные по времени (беру последние 52 недели), то алгоритм предсказания вообще падает с ошибкой. Опять же, в чем причина, не понимаю. Может быть, нельзя обрезать только трейн, надо полностью сначала обрезать первичный датасет, а затем делить на трейн/тест? 